# Tweet # Counts

Time series, counts by week.


#### Libraries + DB

In [1]:
%run utilities.py
%run ../../src/secret_key.py

# for hashtag count summation
import numpy as np

import psycopg2 as pg
import pandas.io.sql as psql

from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
## get connected to the database
conn_str = "dbname='immigration' user='rlrson' password={}".format(pw)
conn = pg.connect(conn_str)

#### Time Periods

Caglar has broken them down into before/after the 'epoch' date. Lets pull them in.

In [3]:
time_periods = pd.read_sql("SELECT * FROM timeinterval;", con=conn)
time_periods

,id,startinterval,endinterval
0,1,2016-12-27 10:00:00+00:00,2017-01-27 10:00:00+00:00
1,2,2017-01-28 10:00:00+00:00,2017-02-27 10:00:00+00:00
2,3,2017-01-21 10:00:00+00:00,2017-01-27 10:00:00+00:00
3,4,2017-01-28 10:00:00+00:00,2017-02-03 10:00:00+00:00
4,5,2017-01-26 10:00:00+00:00,2017-01-27 10:00:00+00:00
5,6,2017-01-28 10:00:00+00:00,2017-01-29 10:00:00+00:00


In [4]:
time_periods.dtypes

id                             int64
startinterval    datetime64[ns, UTC]
endinterval      datetime64[ns, UTC]
dtype: object

In [5]:
# set variables so we can specify a beginning/end for our SQL queries
start = time_periods.iloc[0]['startinterval'].tz_convert(None)
end = time_periods.iloc[1]['endinterval'].tz_convert(None)

#### Week Categorization

We need to create a mapping that will return the week number for us. We'll start at the epoch and work our way backwards week by week. After a month we'll do 'after' the epoch.

In [6]:
# set the time variables
# epoch_start - (delta_week * 4) >> Timestamp('2016-12-30 10:00:00')
# start >> Timestamp('2016-12-27 10:00:00+0000', tz='UTC')

# Epoch
epoch_start = pd.to_datetime('2017-01-27 10:00:00+00:00', utc=True).tz_convert(None)
epoch_end = pd.to_datetime('2017-01-28 10:00:00+00:00', utc=True).tz_convert(None)

# delta
delta_week = pd.Timedelta('7 days')

# before
four_before = epoch_start - (delta_week * 4)
three_before = epoch_start - (delta_week * 3)
two_before = epoch_start - (delta_week * 2)
one_before = epoch_start - (delta_week * 1)

# after
four_after = epoch_end + (delta_week * 4)
three_after = epoch_end + (delta_week * 3)
two_after = epoch_end + (delta_week * 2)
one_after = epoch_end + (delta_week * 1)


def get_week_number(week):
    try:
        week = pd.to_datetime(week, infer_datetime_format=True)
    
        # catch, before/after?
        if (week < start) or (week > end):
            return None

        if (week < four_before):
            return -5
        elif (week < three_before):
            return -4
        elif (week < two_before):
            return -3
        elif (week < one_before):
            return -2
        elif (week < epoch_start):
            return -1
        elif (week < epoch_end):
            return 0
        elif (week < one_after):
            return 1
        elif (week < two_after):
            return 2
        elif (week < three_after):
            return 3
        elif (week < four_after):
            return 4
        else:
            return 5
    except Exception as e:
        print (e, week)
        return None

#### Tweets

We need a count of tweets from every county, for every week (8: 4-before/after). The `tweet` table has tweetid, tweet, date, userid, id, lon, lat, cntyid.

In [7]:
query_test = "SELECT tweetID, date::date, tweet  FROM tweet LIMIT 10;"
test_df = pd.read_sql(query_test, con=conn)

test_df.head(10)

,tweetid,date,tweet
0,819736256682803201,2017-01-12,الذكر عند الخروج من المنزل:بسم الله توكَّلْتُ ...
1,819736257949528065,2017-01-12,اللهم أعذنا من عذاب القبر وعذاب جهنم ♻️
2,819736261476970497,2017-01-12,الذكر عند الخروج من المنزل:بسم الله توكَّلْتُ ...
3,819736261971869702,2017-01-12,ربي أعوذ بك من الكسل وسوء الكبر ♻️
4,819736262655569921,2017-01-12,اللهم أعذنا من عذاب القبر وعذاب جهنم ♻️
5,819736264475865088,2017-01-12,اللهم اهدني فيمن هديت وعافني فيمن عافيت وتولني...
6,819736266711461893,2017-01-12,اللهم اهدني فيمن هديت وعافني فيمن عافيت وتولني...
7,819736266832891904,2017-01-12,RT : Also Pl Ask Ur Muslim Bros 2 Care 4 Hind...
8,819736272369500160,2017-01-12,اللهم أعذنا من عذاب القبر وعذاب جهنم ♻️
9,819736274194038784,2017-01-12,ربي أعوذ بك من الكسل وسوء الكبر ♻️


In [8]:
tweet_query = "SELECT date::date, tweet FROM tweet LIMIT 75000;"
tweet_df = pd.read_sql(tweet_query, con=conn)

tweet_df.head(2)

,date,tweet
0,2017-01-12,الذكر عند الخروج من المنزل:بسم الله توكَّلْتُ ...
1,2017-01-12,اللهم أعذنا من عذاب القبر وعذاب جهنم ♻️


In [9]:
tweet_df.dtypes

date     object
tweet    object
dtype: object

### Hashtags

In [10]:
# https://stackoverflow.com/questions/6331497/an-elegant-way-to-get-hashtags-out-of-a-string-in-python
def get_hashtags(text):
    try:
        tags = set([item.strip("#.,-\"\'&*^!").lower() 
                    for item in text.split() 
                    if ((item.startswith("#") == True) 
                        and (item.startswith("...") == False) 
                        and (item.endswith("...") == False) 
                        and (item.startswith("…") == False) 
                        and (item.endswith("…") == False) 
                        and len(item) < 256)])

        if bool(tags):
            return list(tags)
        else:
            return None
    except Exception as e:
        print (e, text)
        return None

In [11]:
tweet_df['tags'] = tweet_df.tweet.apply(get_hashtags)

tweet_df.head()

,date,tweet,tags
0,2017-01-12,الذكر عند الخروج من المنزل:بسم الله توكَّلْتُ ...,None
1,2017-01-12,اللهم أعذنا من عذاب القبر وعذاب جهنم ♻️,None
2,2017-01-12,الذكر عند الخروج من المنزل:بسم الله توكَّلْتُ ...,None
3,2017-01-12,ربي أعوذ بك من الكسل وسوء الكبر ♻️,None
4,2017-01-12,اللهم أعذنا من عذاب القبر وعذاب جهنم ♻️,None


In [53]:
# filter the dataframe to only those tweets with hashtags
test = tweet_df[tweet_df['tags'].isnull() == False][['date', 'tags']]

In [74]:
test_day = test.groupby('date').apply(group_month_tags).reset_index()
test_day.columns = ['date', 'tags']

In [76]:
test_day.head()

,date,tags
0,2017-01-12,"[spirit212, jihad, ahokermakinkalap, liberals,..."
1,2017-01-13,"[followback, followback, isis, islam, quran, t..."
2,2017-01-14,"[islam, islam, quran, christianity, art, music..."


In [84]:
so_far = count_tweets(tweet_df)
so_far.head()

,date,tags
38,2017-01-12,"[spirit212, jihad, ahokermakinkalap]"
54,2017-01-12,[liberals]
269,2017-01-12,"[spirit212, jihad, ahokermakinkalap]"
315,2017-01-12,[muslim]
414,2017-01-12,[rapefugees]
3192,2017-01-14,[islam]
14004,2017-01-13,[followback]
14027,2017-01-13,[followback]
14352,2017-01-13,"[isis, islam, quran, taliban, alnusra, alqaeda]"
14492,2017-01-13,"[iraq, lebanon, egypt, bahrain, syria, jordan]"


In [86]:
type(so_far.apply(group_month_tags))

pandas.core.series.Series

#### Parallelize

This is going to take forever unless we do.

In [68]:
from multiprocessing import Pool  # faster

def parallelize_series(series, func):
    pool = Pool(6)

    df = pool.map(func, series)

    pool.close()
    pool.join()
    return df

def group_month_tags(dataframe):
    """
    Takes a Pandas Dataframe[date, tags] and returns a concatentation of them all
    """
    return np.concatenate(dataframe.tags.values)

def count_month_tags(grouped_series, day):
    counts = pd.value_counts(grouped_series)
    
    counts = counts.reset_index()
    counts.columns = ['tag', 'count']
    
    counts['date'] = day
    # rename here
    return counts

#### DataFrame Construction

Putting together everything we've done so far:
1. Read in the tweets
2. Calculate the hashtags variable column
3. Filter the dataframe to only those with hashtags
4. GroupBy the `date` column, and concatenate all of the hashtags within that day
5. Get value counts of the hashtags as a dataframe
6. add to the regular.

In [82]:
def count_tweets(tweets):
    # 2
    #tweets['tags'] = tweets['tweet'].apply(get_hashtags)
    tweets['tags'] = parallelize_series(tweets['tweet'].values, get_hashtags)
    
    # 3 
    tweets = tweets[tweets['tags'].isnull() == False]
    
    # 4
    by_day = tweets[['date', 'tags']].groupby('date')#.apply(group_month_tags)
    
    # 5
    counts = pd.DataFrame()
    #for date, tags in by_day:
        #all_tags = group_month_tags(tags)
        #temp_counts = pd.value_counts(all_tags)
    return by_day

In [66]:
# df = pd.DataFrame()
# for chunk in pd.read_sql('select * from table_name', con=conn, chunksize=5000):
#    df = df.append(chunk)

# fake_query to test
counts = pd.DataFrame()
for chunk in pd.read_sql(tweet_query, con=conn, chunksize=5000):
    formatted = group_tweets(chunk)
    counts = counts.append(formatted)

#### Actual Counts!!!

In [70]:
# for real this time
full_query = ""

st_cnts = pd.DataFrame()
for chunk in pd.read_sql(full_query, con=conn, chunksize=50000):
    formatted = group_tweets(chunk)
    st_cnts = st_cnts.append(formatted)

In [82]:
# work here

In [74]:
final.head()

,st,week,cnty
0,AL,-2.0,204
1,AL,-1.0,2103
2,AL,1.0,49415
3,AL,2.0,23908
4,AL,4.0,6939


In [75]:
# finally
f = name_file_path('state-tweet-counts.csv', processed_web_dir)

final.rename(columns={'cnty': 'cnt'}, inplace=True)
final.to_csv(f, index=False)

---

In [76]:
conn.close()

In [48]:
# Wide cells
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

# number of text characters per column
pd.set_option('max_colwidth', 140)